# Lab 4 - Disk Imaging Part of the Lab

- Jupyter Notebook with image acquisition tools discussed in class, dd, dcfldd, ewfacquire, aff4imager (pmem tool suite), proving integrity of the image and chain of custody with signing

**Note:** The drive must be attached to this host machine to take an image of it. This was done on Kali Linux VM rather than the typical WIN-FOR configured machine used for the labs.

# Mounting

In this notebook, the typical WIN-FOR configured machine's VMWare virtual disk is added to the kali linux as a mounted device for forensic imaging.

In [3]:
%%bash
lsblk

NAME   MAJ:MIN RM   SIZE RO TYPE MOUNTPOINTS
sda      8:0    0   200G  0 disk 
├─sda1   8:1    0 189.8G  0 part /
├─sda2   8:2    0     1K  0 part 
└─sda5   8:5    0  10.2G  0 part [SWAP]
sdb      8:16   0   200G  0 disk 
├─sdb1   8:17   0   300M  0 part 
├─sdb2   8:18   0   100M  0 part 
├─sdb3   8:19   0   128M  0 part 
└─sdb4   8:20   0 199.5G  0 part /mnt/winforvm
sr0     11:0    1   4.3G  0 rom  


In [2]:
%%bash
# sdb4 is the main partition

!mkdir /mnt/winforvm
!mount -o ro /dev/sdb4 /mnt/winforvm

mkdir: cannot create directory ‘/mnt/winforvm’: File exists


# Acquisition
Acquire a RAW image of the Windows VM disk using `dcfldd`, generate acquisition-time SHA-256, and verify integrity.

**NOTE:** For this lab, the wrong image will be taken (sdb1 instead of sdb4) simply because I do not have the drive space available to create such a large forensics image right now. The process, however, would be the exact same for the proper partition.

1) Identify block device backing `/mnt/winforvm`
- Sets case folders and disk device behind /mnt/winforvm. Writes tool version to logs.
2) `dcfldd` with `conv=noerror,sync` and `hash=sha256`
- Uses dcfldd to acquire a RAW image with SHA-256 acquisition hashing and logs the run.
3) Independently hash the image (`sha256sum`) and compare
- Confirm chain-of-integrity for further future analysis.

In [14]:
%%bash

set -euo pipefail

EVID_ROOT="/home/cloudzest/repos/CYBS-5014/lab4/output/"
SRC_MOUNT="/mnt/winforvm"
SOURCE_DEV="/dev/sdb1"
PART_DEV="/dev/sdb"

mkdir -p "$EVID_ROOT"/{logs,images,notes}

echo "Mount point:   $SRC_MOUNT"
echo "Partition dev: $PART_DEV"
echo "Source device: $SOURCE_DEV"

{
  echo "dcfldd version:"
  dcfldd --version 2>&1 || true
} | tee "$EVID_ROOT/logs/tool_versions.txt"

Mount point:   /mnt/winforvm
Partition dev: /dev/sdb
Source device: /dev/sdb1
dcfldd version:
dcfldd (dcfldd) 1.9.3
Written by: dcfldd by Nicholas Harbour and others.
GNU dd by Paul Rubin, David MacKenzie and Stuart Kemp.
Currently, the source code is at
https://github.com/resurrecting-open-source-projects/dcfldd.

Copyright (C) 1985-2006 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.


In [15]:
%%bash
set -euo pipefail

EVID_ROOT="/home/cloudzest/repos/CYBS-5014/lab4/output/"
SRC_MOUNT="/mnt/winforvm"
SOURCE_DEV="/dev/sdb1"
PART_DEV="/dev/sdb"
TS="$(date -u +'%Y%m%dT%H%M%SZ')"

# Output names
IMG_PATH="$EVID_ROOT/images/lab4_${TS}.raw"
HASH_LOG="$EVID_ROOT/images/lab4_${TS}.raw.sha256.acq.txt"
DD_LOG="$EVID_ROOT/logs/lab4_${TS}_dcfldd.log"

echo "Imaging from: ${SOURCE_DEV}"
echo "Image path:   ${IMG_PATH}"

# Perform acquisition with hashing. The hash recorded is over the input stream.
# conv=noerror,sync continues on read errors and pads blocks to keep offsets aligned.
sudo dcfldd if="${SOURCE_DEV}" of="${IMG_PATH}" conv=noerror,sync hash=sha256 hashlog="${HASH_LOG}" status=on 2>&1 | tee "${DD_LOG}"

echo "Acquisition complete."
echo "Acquisition hash log at: ${HASH_LOG}"

Imaging from: /dev/sdb1
Image path:   /home/cloudzest/repos/CYBS-5014/lab4/output//images/lab4_20251110T001613Z.raw
9472 blocks (296Mb) written.
9600+0 records in
9600+0 records out
Acquisition complete.
Acquisition hash log at: /home/cloudzest/repos/CYBS-5014/lab4/output//images/lab4_20251110T001613Z.raw.sha256.acq.txt


In [21]:
%%bash
set -euo pipefail

EVID_ROOT="/home/cloudzest/repos/CYBS-5014/lab4/output/"

# Re-identify the most recent image and hash files
IMG_PATH="$(ls -1t "$EVID_ROOT/images"/*.raw | head -n1)"
HASH_LOG="$(ls -1t "$EVID_ROOT/images"/*.raw.sha256.acq.txt | head -n1)"
VERIFY_LOG="${IMG_PATH}.sha256.verify.txt"

echo "Verifying image: ${IMG_PATH}"
sha256sum "${IMG_PATH}" | tee "${VERIFY_LOG}"

echo
echo "---- Acquisition-time hash (from dcfldd) ----"
cat "${HASH_LOG}" | grep -Po '\(sha256\):\s*\K\S+'
echo "---- Verification hash (sha256sum) ----"
cat "${VERIFY_LOG}" | grep -Po '.*?(?=\s)'

# Simple compare
ACQ_HASH="$(awk '{print $1}' "${HASH_LOG}" | tail -n1)"
VER_HASH="$(awk '{print $1}' "${VERIFY_LOG}")"

Verifying image: /home/cloudzest/repos/CYBS-5014/lab4/output//images/lab4_20251110T001613Z.raw
0c92fe57f82fcdf86c643c59dea536b2bbca64823a0bc738b7287ba2e4d1e8d5  /home/cloudzest/repos/CYBS-5014/lab4/output//images/lab4_20251110T001613Z.raw

---- Acquisition-time hash (from dcfldd) ----
0c92fe57f82fcdf86c643c59dea536b2bbca64823a0bc738b7287ba2e4d1e8d5
---- Verification hash (sha256sum) ----
0c92fe57f82fcdf86c643c59dea536b2bbca64823a0bc738b7287ba2e4d1e8d5
